In [1]:
import os
import json
import base64
import io
import time
from PIL import Image
from openai import OpenAI

In [1]:
import secrets
secrets.token_hex(24)

'5e67649ca7508d280ac840f20dc2982a2ad0443a64420488'

In [2]:
# ==============================================================================
#  !! 关键步骤 1: 配置你的大模型 API !!
#  请将这部分从 a_ocr_config.py 复制过来，并确保填写正确。
# ==============================================================================
LLM_CONFIG = {
    # 示例: OpenAI 官方 (GPT-4o)
    "API_KEY": "sk-udzsogssqinpcokdsgxehmopfobqycpwflgtzarmdhqtcuio",  # <--- 在这里填入你的API密钥
    "BASE_URL": "https://api.siliconflow.cn/v1", # <--- 如果使用官方API，请用这个
    "MODEL_NAME": "Qwen/Qwen2.5-VL-72B-Instruct" #  "Qwen/QVQ-72B-Preview",
}

# ==============================================================================
#  !! 关键步骤 2: 设计你的 JSON 提取指令 (Prompt) !!
#  将这部分从 a_ocr_config.py 复制过来。
# ==============================================================================
JSON_PROMPT = """
你是一个顶级的视觉识别能力（OCR）专家。
你的任务是严格按照用户提供的图片，提取所有关键数据指标，并以一个纯净的、不含任何额外解释的JSON格式返回。

请根据图片内容，填充以下JSON结构。
- 对于图片中存在的指标，请精确提取其数值。
- 对于数值，只返回数字（整数或浮点数），不要包含任何单位如 "¥"、"%" 或 "分秒"。对于百分比，请直接返回数字，例如 "12.5%" 应返回 12.5。对于时间 "1分6秒"，请计算总秒数后返回。
- 对于图片中不存在或无法识别的指标，请使用 `null` 作为其值。
- 最终的输出必须是一个能被直接解析的JSON对象，不要在JSON代码块前后添加任何文字、注释或markdown标记(如```json ```)。

这是你必须严格遵守的JSON结构：
{
  "live_start_time": "HH:MM:SS 或 HH:MM 格式的字符串",
  "gmv": "浮点数",
  "gpm": "浮点数",
  "order_count": "整数",
  "buyer_count": "整数",
  "vv": "整数",
  "avg_online_users": "整数",
  "avg_watch_time_seconds": "整数 (总秒数)",
  "new_followers": "整数",
  "click_to_order_ratio": "浮点数 (例如 12.5)",
  "view_to_order_ratio": "浮点数",
  "expose_to_view_ratio": "浮点数",
  "view_to_interact_ratio": "浮点数",
  "follower_order_ratio": "浮点数"
}
"""

# --- 以下是需要测试的核心函数，直接从 app.py 复制过来 ---

def image_to_base64(image: Image.Image, format="PNG") -> str:
    """
    将Pillow图片对象转换为Base64字符串。
    【增强版】: 在保存前，强制将图片转换为RGB模式，以确保最大兼容性。
    """
    buffered = io.BytesIO()
    
    # --- 核心修改 ---
    # 无论原始图片是什么模式 (L, RGBA, CMYK等), 
    # 都统一转换为'RGB'模式。这可以解决绝大多数格式兼容性问题。
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    # 现在图片保证是'RGB'模式，可以安全地保存为JPEG或PNG
    save_format = format.upper()
    if save_format == 'JPEG':
        image.save(buffered, format="JPEG")
        mime_type = "image/jpeg"
    else: # 默认或指定PNG
        image.save(buffered, format="PNG")
        mime_type = "image/png"

    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return f"data:{mime_type};base64,{img_str}"

In [3]:
# --- 主测试逻辑 ---
# 1. 提示用户输入要测试的图片路径
image_path = input("请输入要测试的图片文件路径 (例如: 1.jpg 或 D:\\images\\test.png): ")

请输入要测试的图片文件路径 (例如: 1.jpg 或 D:\images\test.png):  1.png


In [ ]:
api_client = OpenAI(api_key=LLM_CONFIG["API_KEY"], base_url=LLM_CONFIG["BASE_URL"])
test_image = Image.open(image_path)
base64_image = image_to_base64(test_image)

response = api_client.chat.completions.create(
    model=LLM_CONFIG["MODEL_NAME"],
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": JSON_PROMPT},
                {"type": "image_url","image_url": {"url": base64_image},
                }
            ],
        }
    ],
    max_tokens=2048,
    temperature=0.0,
    #stream=True
)

json_string = response.choices[0].message.content
json_string